# Bin Packing Lab

- Authors:
  - Luigi Quattrociocchi (quattrl@mcmaster.ca)
  - Dennis Fong (fongd1@mcmaster.ca)
- Group ID on Avenue: 42
- Gitlab URL: https://gitlab.cas.mcmaster.ca/quattrl/l2-bin-packing

## How to use the provided code?

_(this section is just here for information, you can get rid of it in your own report)_

In [1]:
from macpacking.reader import DatasetReader, BinppReader, JBurkardtReader
from macpacking.model  import Online, Offline
import macpacking.algorithms.offline as offline

Now that the business code is imported, we can load an existing dataset

In [2]:
# dataset = '_datasets/binpp/N1C1W1/N1C1W1_B.BPP.txt'
# dataset = '_datasets/binpp-hard/HARD0.BPP.txt'
# reader: DatasetReader = BinppReader(dataset)
# print(f'Dataset: {dataset}')
# print(f'  - Bin Capacity: {reader.offline()[0]}')
# print(f'  - Objects to pack: {sorted(reader.offline()[1])}')

dataset_capacity = '_datasets/jburkardt/p01_c.txt'
dataset_weights = '_datasets/jburkardt/p01_w.txt'
reader: DatasetReader = JBurkardtReader(dataset_capacity, dataset_weights)
print(f'Dataset: {dataset_capacity}, {dataset_weights}')
print(f'  - Bin Capacity: {reader.offline()[0]}')
print(f'  - Objects to pack: {sorted(reader.offline()[1])}')

Dataset: _datasets/jburkardt/p01_c.txt, _datasets/jburkardt/p01_w.txt
  - Bin Capacity: 100
  - Objects to pack: [3, 7, 11, 33, 33, 33, 50, 60, 70]


Acording to the `oracle.xslx` file, we now that the optimal solution for this case is to use _31_ bins. Let's call the baseline algorithm, which is an offline one, and see how it performs.

In [3]:
import macpacking.algorithms.baseline as baseline
strategy: Offline = baseline.BenMaier()
result = strategy(reader.offline())
print(f'nb_bins = {len(result)}')
print(f'{sorted(result)}')

nb_bins = 4
[[33], [50, 33, 11, 3], [60, 33, 7], [70]]


So the baseline finds the optimal solution. That's good news! Let's call our very own version of `NextFit`, as an offline algorithm.

In [4]:
import macpacking.algorithms.online as online
strategy: Offline = offline.NextFit()
result = strategy(reader.offline())
print(f'nb_bins = {len(result)}')
print(f'{sorted(result)}')

nb_bins = 4
[[33, 33, 11, 7, 3], [50, 33], [60], [70]]


Damn it, this algorithm is 4 bins far from the optimal solution! Let's try an online version. Usually, they perform worst, so let's measure it.

In [5]:
strategy: Online = online.NextFit()
result = strategy(reader.online())
print(f'nb_bins = {len(result)}')
print(f'{sorted(result)}')

nb_bins = 4
[[33, 11, 7, 33, 3], [50, 33], [60], [70]]


As expected, the online version is worst!

In [6]:
strategy: Online = online.TerribleFit()
result = strategy(reader.online())
print(f'nb_bins = {len(result)}')
print(f'{sorted(result)}')

nb_bins = 9
[[3], [7], [11], [33], [33], [33], [50], [60], [70]]


## Task 1

### Explanation of SOLID principles

The given code follows many design practices and implements many classic design patterns. This allows it to fulfil a couple of the SOLID principles.

**S: Single Responsibility**  
Each class has a single responsibility. For instance, the `DatasetReader` class is only responsible for transforming data from the disk into a format usable by other classes. Likewise, each concrete bin packing problem implementation has one responsibility: solving the bin packing problem.

**O: Open-closed**  
The existence of the `Online` and `Offline` abstract classes allow for extentibility in the addition of new algorithms, without requiring modifications of any existing classes. The same reasoning applies for the `DatasetReader` class.

**L: Liskov substitution**  
This is obviously satified with the `Online` and `Offline` base classes and its derived classes; the online and offline bin packing algorithms `NextFit_On`, `NextFit_Off`, and `BenMaier`, can be used as if they were their parent classes `Online` and `Offline`.

**I: Interface Segregation**  
This property is also obviously maintained. Both `BinPacker` subclasses and `DatasetReader` only provide one abstract method each, which every subclass implements. The `BinPacker` interface itself has no methods, as there is currently no shared interface between the online and offline algorithms (the process functions take different sets of parameters).

**D: Dependency Inversion**  
Similarly to the Liskov substitution principle, the abstract classes `Online` and `Offline` can be depended upon rather than their concrete subclasses. This is demonstrated in the Jupyter Notebook, in which online and offline strategies are created and used.

### Explanation of dataset dimensions
There are three important dimensions which we consider in the dataset.
- N: Number of items
- C: Bin capacity
- W: Weight (size) of each item

**Number of items**  
This number represents the total number of items we want to put into bins. This quantity is important because the time and space required to solve the bin packing problem with N items is proportional to N.

**Bin capacity**  
This number represents the capacity (maximum weight) of each bin. This quantity is important because the number of bins required in the optimal solution will be inversely proportional to the bin capacity. Some algorithms can take advantage of small capacities to come closer to an optimal solution.

**Weight of each item**  
This number represents the weight (or size) of each item. This quantity is important because the weight of each bin (relative to the bin capacity) can dictate the number of bins required in the optimal solution. Additionally, the weight of the bins can impact the optimality of certain algorithms.


## Self-reflection questions

As part of the self-reflection dimension of an experiential course, each member of the group is expected to answer to the following four questions:

  - What process did you go through to produce this result? (Backward)
  - What were your standards for this piece of work? Did you meet your standards? (Inward)
  - What the one thing you particularly want people to notice when they look at your work? (Outward)
  - What lessons will you keep from this reading/lecture in your professional practice? (Forward)